In [ ]:
from labwons.util import DATETIME, DD
import os

ACTION = os.environ.get("GITHUB_EVENT_NAME", "LOCALHOST").upper()
HOSTID = os.environ.get("USERDOMAIN", "COLAB") if ACTION == "LOCALHOST" else "GITHUB"
JOBS   = [
    # "FETCH.AFTER-MARKET"
]

# GITHUB ACTION
if ACTION == "SCHEDULE":

    # SYSTEM EXITS WHEN THE LATEST TRADING DATE AND BUILD DATE IS DIFFERENT.
    # THIS CODE IS IMPLEMENTED IN ORDER TO AVOID RUNNING ON WEEKDAY OR HOLIDAY
    # OF THE MARKET.
    if DATETIME.TRADING != DATETIME.TODAY:
        raise SystemExit
    
    # IF THE TASK IS TRIGGERED BEFORE THE MARKET CLOSES, WHICH HARDLY HAPPENS,
    # SYSTEM PAUSES UNTIL THE MARKET CLOSES
    while (DATETIME.TIME().hour == 15) and (15 <= DATETIME.TIME().minute < 31):
        DATETIME.pause(30)
    
    if DATETIME.TIME().hour >= 20:
        JOBS = ["FETCH.ECOS", "FETCH.FRED", "FETCH.FINANCIAL-STATEMENT"]
    else:
        JOBS = ["FETCH.AFTER-MARKET", "FETCH.STOCKS"]
    JOBS += ["BUILD.MARKET-BASELINE", "BUILD.MACRO-BASELINE", "DEPLOY.ALL"]

# if ACTION == "LOCALHOST" and HOSTID == "HKEFICO":
#     JOBS = [job for job in JOBS if not job.startswith("FETCH")]

In [ ]:
from labwons.fetch import FETCH_OK, AfterMarket
from labwons.path import PATH


archive = os.path.join(PATH.ARCHIVE, DATETIME.TODAY)
dataset = DD()
if "FETCH.AFTER-MARKET" in JOBS:    
    afterMarket = AfterMarket()
    if afterMarket.status == FETCH_OK:
        dataset[os.path.join(archive, afterMarket.fname)] = afterMarket.data


# FETCH 기록은 날짜별로 아카이빙 하자 Diff 할 필요가 없고 불필요한 이력 용량을 만들지 않아야지
# 일정 기간이 지난 날짜 폴더들은 드라이브로 백업해서 용량을 관리하는거야
# 여기 섹션에 모든 FETCH를 모으고 최종적으로 데이터프레임들을 날짜로 된 폴더에 넣는다
# FETCH를 하지 않는 경우 베이스라인에서 참조할 수 있도록 최근 날짜를 찾아서 경로로 전달한다!

os.makedirs(archive, exist_ok=True)
for file, data in dataset.items():
    data.to_parquet(f"{file}.parquet", engine='pyarrow')

[FETCH] 2025-08-22 10:29:31,717 [INFO] RUN [FETCH AFTER MARKET PYKRX DATA]
[FETCH] 2025-08-22 10:29:31,787 [ERROR] >>> FAILED TO FETCH TRADING DATE: index -1 is out of bounds for axis 0 with size 0


In [5]:
from labwons.util import Mail

log = ''
for file in ('fetch', 'build'):
    with open(os.path.join(PATH.LOGS, rf'{file}_{DATETIME.TODAY}.log'), 'r', encoding='utf-8') as f:
        log += f.read()
    log += '\n'

# report = Mail()
# report.content = 
# report.subject = 
log

ImportError: cannot import name 'Mail' from 'labwons.util' (E:\$proj\labwons\src\labwons\util.py)